In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
!pip install -U datasets
!pip install fsspec==2023.9.2
!pip install datasets
!pip install peft
!pip install -U bitsandbytes


In [ ]:
from datasets import load_dataset

# Load the vqa-rad dataset
dataset = load_dataset("mdwiratathya/SLAKE-vqa-english")


In [ ]:
import torch
import io
import pickle
import pandas as pd
from transformers import BlipProcessor, BlipForQuestionAnswering
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoProcessor, AutoModelForVision2Seq,AutoModel
from peft import LoraConfig  # Import LoRA configuration
from transformers import AutoModelForCausalLM

# Check if LoRA or QLoRA should be used

    # Define LoRA configuration
lora_config = LoraConfig(
r=4,
lora_alpha=4,
lora_dropout=0.1,
target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Adjust based on inspected names
bias="none",
init_lora_weights="gaussian"
)

    # If using QLoRA, configure Bits and Bytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


    # Load model with quantization configuration if using QLoRA
from transformers import AutoProcessor, AutoModelForPreTraining

processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = AutoModelForPreTraining.from_pretrained("llava-hf/llava-1.5-7b-hf",
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
)

# Add LoRA configuration to the model
model.add_adapter(lora_config)
model.enable_adapters()


# Move model to the correct device


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/674 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mdwiratathya/SLAKE-vqa-english")
dataset['train'] = dataset['train'].filter(lambda example: example['question'] is not None and example['answer'] is not None and example['image'] is not None)
dataset['test'] = dataset['test'].filter(lambda example: example['question'] is not None and example['answer'] is not None and example['image'] is not None)
dataset['validation'] = dataset['validation'].filter(lambda example: example['question'] is not None and example['answer'] is not None and example['image'] is not None)


Filter:   0%|          | 0/4919 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1061 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1053 [00:00<?, ? examples/s]

In [ ]:
# Inspect the dataset for None values
for idx, example in enumerate(dataset['test']):
    if example['question'] is None or example['image'] is None or example['answer'] is None:
        print(f"Missing values in example at index {idx}: {example}")


In [ ]:
from datasets import load_dataset
from torch.utils.data import Dataset
from PIL import Image
import random
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

# Define the VQADataset class
class VQADataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        image = self.dataset[idx]['image']  # Assuming it's a PIL image

        return {
            "image": image,
            "query": {"en": question},
            "answers": [answer]
        }


# Define the MyDataCollator class
class MyDataCollator:
    def __init__(self, processor):
        self.processor = processor
        self.image_token = '<image>'
        self.end_of_utterance_token = '<end_of_utterance>'
        self.pad_token_id = getattr(processor, 'pad_token_id', 0)  # Default to 0 if not found

    def __call__(self, examples):
        texts = []
        for example in examples:
            question = example["query"]['en']
            answer = random.choice(example["answers"])

            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Answer briefly."},
                        {"type": "text", "text": self.image_token},
                        {"type": "text", "text": question}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": answer},
                        {"type": "text", "text": self.end_of_utterance_token}
                    ]
                }
            ]

            text = self.processor.apply_chat_template(messages, add_generation_prompt=False)

            if isinstance(text, list):
                text = " ".join(str(item) for item in text)
            else:
                text = str(text)

            texts.append(text.strip())

        text_batch = self.processor(text=texts, return_tensors="pt", padding=True)

        # Prepare the batch dictionary
        batch = {
            "input_ids": text_batch["input_ids"],
            "attention_mask": text_batch["attention_mask"]
        }

        # Create labels
        labels = batch["input_ids"].clone()
        labels[labels == self.pad_token_id] = -100  # Ignore padding in loss calculation
        batch["labels"] = labels

        return batch




# Create dataset objects for each split
train_dataset = VQADataset(dataset=dataset['train'], processor=processor)
test_dataset = VQADataset(dataset=dataset['test'], processor=processor)
valid_dataset = VQADataset(dataset=dataset['validation'], processor=processor)

# Data collator for batching
data_collator = MyDataCollator(processor=processor)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "LLavaMed",
    learning_rate = 2e-4,
    fp16 = True,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 8,
    dataloader_pin_memory = False,
    save_total_limit = 3,
    eval_strategy ="steps",
    save_strategy = "steps",
    eval_steps = 100,
    save_steps = 100,
    max_steps = 500,
    logging_steps = 5,
    remove_unused_columns = False,
    push_to_hub=False,
    label_names = ["labels"],
    load_best_model_at_end = False,
    report_to = "none",
    optim = "paged_adamw_8bit",
    weight_decay=0.05
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset
)

In [ ]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,0.571300,0.521260
200,0.444400,0.410863
300,0.402900,0.378802
400,0.328500,0.353476


Step,Training Loss,Validation Loss
100,0.571300,0.521260
200,0.444400,0.410863
300,0.402900,0.378802
400,0.328500,0.353476
500,0.310600,0.344394


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error: Unauthorized for url: https://huggingface.co/llava-hf/llava-1.5-7b-hf/resolve/main/config.json (Request ID: Root=1-686e893f-566d577477ce72465b566bd9;d5e41e54-635b-4c84-a375-f5a5bb54c628)

Invalid credentials in Authorization header - silently ignoring the lookup for the file config.json in llava-hf/llava-1.5-7b-hf.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in llava-hf/llava-1.5-7b-hf - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=500, training_loss=0.5726375994682312, metrics={'train_runtime': 2950.1615, 'train_samples_per_second': 1.356, 'train_steps_per_second': 0.169, 'total_flos': 5932480740550656.0, 'train_loss': 0.5726375994682312, 'epoch': 0.8131734092295182})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,LlavaProcessor

#32
save_directory = "/content/drive/MyDrive/MultiModal/VQAForPublish/models/R4llavaFinalslakee"

from transformers import AutoConfig

config = AutoConfig.from_pretrained("llava-hf/llava-1.5-7b-hf")  # Use the base model
config.save_pretrained(save_directory)
model.save_pretrained(save_directory)
# processor = LlavaProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
processor.save_pretrained(save_directory)

print(f"✅ Model saved at: {save_directory}")

✅ Model saved at: /content/drive/MyDrive/MultiModal/VQAForPublish/models/R4llavaFinalslakee


In [ ]:
from huggingface_hub import HfApi

# Set up the model and tokenizer save path
save_directory = "/content/drive/MyDrive/MultiModal/VQAForPublish/models/R4llavaFinalslakee"
repo_name = "R4llava9JSlakF"
user_name = "Dtarget"

api = HfApi()

# Create the repository as private
api.create_repo(repo_id=f"{user_name}/{repo_name}", private=True, repo_type="model")

api.upload_folder(
    folder_path=save_directory,
    repo_id=f"{user_name}/{repo_name}",
    path_in_repo=""  # Upload to root of the repository
)

print(f"✅ Model uploaded to Hugging Face privately: https://huggingface.co/{user_name}/{repo_name}")


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/19.2M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

✅ Model uploaded to Hugging Face privately: https://huggingface.co/Dtarget/R4llava9JSlakF


In [ ]:


# Evaluate the model
results = trainer.evaluate()

print(results)


{'eval_loss': 0.34439367055892944, 'eval_runtime': 260.461, 'eval_samples_per_second': 4.043, 'eval_steps_per_second': 4.043, 'epoch': 0.8131734092295182}


In [ ]:
test_results = trainer.evaluate(test_dataset)

print(test_results)

{'eval_loss': 0.34057193994522095, 'eval_runtime': 235.4565, 'eval_samples_per_second': 4.506, 'eval_steps_per_second': 4.506, 'epoch': 0.8131734092295182}
